# Load Packages & Set Working Directory

In [ ]:
# packages

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import csv
import json
import re
import simpledorff
import pandas as pd
import transformers
from transformers import AutoTokenizer
from transformers import  LlamaForCausalLM, LlamaTokenizer, pipeline
import transformers

import torch
from torch import cuda, bfloat16, manual_seed

from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.llms import HuggingFacePipeline

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score
from sklearn.metrics import classification_report


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
torch.clear_autocast_cache()

In [ ]:
os.getcwd()

# go one level up in the directory
os.chdir("/data/storage100gb5/NOS")

huggingface_cache_dir = 'model'

# change huggingface cache
os.environ['TRANSFORMERS_CACHE'] = huggingface_cache_dir

# Load the Model from Huggingface

In [ ]:
torch.manual_seed(0)

model_id = 'berkeley-nest/Starling-LM-7B-alpha'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    cache_dir=huggingface_cache_dir
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    cache_dir=huggingface_cache_dir
)
model.eval()

print(f"Model loaded on {device}")

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    cache_dir=huggingface_cache_dir)

In [ ]:
device = torch.device('cuda')
print("GPU Name:", torch.cuda.get_device_name(device))
print("Memory Usage:", torch.cuda.memory_allocated(device) / 1024 ** 3, "GB")
print("Max Memory Usage:", torch.cuda.max_memory_allocated(device) / 1024 ** 3, "GB")

In [ ]:
torch.manual_seed(0)
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.5,  # 'randomness' of outputs, 0.0 is not possible, so use a very small number
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  
)

llm = HuggingFacePipeline(pipeline=generate_text)

## Helper functions

In [ ]:
def regex_extract_to_dataframe(strings):
    # Initialize empty lists to store extracted values
    article_ids = []
    about_covid_values = []

    # Define regex pattern for article_id and about_covid with optional double quotes
    article_id_pattern = r'"article_id"\s*:\s*"?(\d+)"?'
    about_covid_pattern = r'"about_covid"\s*:\s*"?(\d)"?'

    # Iterate through each string
    for string_data in strings:
        # Use regex to find matches for article_id
        article_id_match = re.search(article_id_pattern, string_data)

        # Use regex to find matches for about_covid
        about_covid_match = re.search(about_covid_pattern, string_data)

        # Extract values from the regex matches
        article_id = int(article_id_match.group(1)) if article_id_match else None
        about_covid = int(about_covid_match.group(1)) if about_covid_match else None

        # Append values to the respective lists
        article_ids.append(article_id)
        about_covid_values.append(about_covid)

    # Create a DataFrame using the extracted values
    df = pd.DataFrame({
        "article_id": article_ids,
        "about_covid": about_covid_values
    })

    return df


In [ ]:
def zero_shot_prompt_messages(system_prompt, input_prompt, main_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": input_prompt + "\n" + main_prompt},
    ]
    prompt = generate_text.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

In [ ]:
def few_shot_prompt_messages(system_prompt, input_prompt, main_prompt, examples):
    messages = []
    messages.append({"role": "system", "content": system_prompt})

    for user_prompt, assistant_prompt in examples:
        messages.append({"role": "user", "content": user_prompt})
        messages.append({"role": "assistant", "content": assistant_prompt + "\n"})

    messages.append({"role": "user", "content": input_prompt + "\n" + main_prompt})

    prompt = generate_text.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

# Data Prep

## Get Annotated NOS Articles DF

In [ ]:
df = pd.read_csv('data/nos_analysis/nos_llm_analysis_final.csv',
                 sep = ';', encoding = 'utf-8', quoting=csv.QUOTE_NONNUMERIC)

# change article_id to integer
df['article_id'] = df['article_id'].astype(int)

topic_vars = ['about_covid',  'topic_a', 'topic_b', 'topic_c', 'topic_d', 'topic_e', 'topic_f', 'topic_g', 'topic_h', 
              'topic_i', 'topic_j', 'topic_k', 'topic_l', 'topic_m', 'topic_n', 'topic_o']

# change all topic vars to int
for i in topic_vars:
    df[i] = df[i].astype(int)

# remove line break
df['Text'] = df['Text'].str.replace('[LINE_BREAK]', '\n ')

print(df.shape)
df.head()


In [ ]:
# are there duplicate articles? 
duplicates = df[df.duplicated(subset=['article_id'], keep=False)]
duplicates

In [ ]:
len(df)

## Get Examples 

Randomly selected examples from the annotated dataset, that is not part of the dev or test sets

In [ ]:
df_covid = pd.read_csv('data/nos_analysis/examples/about_covid.csv', sep = ';', encoding = 'utf-8', quoting=csv.QUOTE_NONNUMERIC)

# change article_id to integer
df_covid['article_id'] = df_covid['article_id'].astype(int)

topic_vars = ['about_covid',  'topic_a', 'topic_b', 'topic_c', 'topic_d', 'topic_e', 'topic_f', 'topic_g', 'topic_h', 
              'topic_i', 'topic_j', 'topic_k', 'topic_l', 'topic_m', 'topic_n', 'topic_o']

# change all topic vars to int
for i in topic_vars:
    df_covid[i] = df_covid[i].astype(int)

# remove line break
df_covid['Text'] = df_covid['Text'].str.replace('[LINE_BREAK]', '\n ')

print(df_covid.shape)


df_covid.head()

In [ ]:
print(df_covid.article_id.value_counts())
print(df_covid.Text.values)
print(df_covid.Keywords.values)
print(df_covid.Category.values)

In [ ]:
not_covid = pd.read_csv('data/nos_analysis/examples/not_covid.csv', sep = ';', encoding = 'utf-8', quoting=csv.QUOTE_NONNUMERIC)

# change article_id to integer
not_covid['article_id'] = not_covid['article_id'].astype(int)

# change all topic vars to int
for i in topic_vars:
    not_covid[i] = not_covid[i].astype(int)

# remove line break
not_covid['Text'] = not_covid['Text'].str.replace('[LINE_BREAK]', '\n ')

print(not_covid.shape)
not_covid.head()

In [ ]:
not_covid.article_id.value_counts()

In [ ]:
print(not_covid.article_id.value_counts())
print(not_covid.Text.values)
print(not_covid.Keywords.values)
print(not_covid.Category.values)

## Prompt building

In [ ]:
system_prompt = """
As a helpful AI assistant, your task is to determine the main topic of news articles. Articles may focus on either the "Coronavirus and/or the COVID-19 pandemic" or some other topic.
A main topic is the overarching theme discussed in the majority of the news article. For an article to have the main topic of the "Coronavirus and/or the COVID-19 pandemic", it should predominantly discuss these subjects.
"""

input_prompt = """
Read the following article with the ID {article_id}: {text} \n
This article falls under the categories: {category} and contains the keywords: {keywords}.
"""

main_prompt = """
Take a moment to understand the article. 
Remember, for a topic to be a main topic of the news article, it should be discussed in the majority of the article. 

Based on the information provided, determine if the main topic of this news article is the "Coronavirus and/or the COVID-19 pandemic" or another subject. 
Assign a value of 1 if the main topic is the "Coronavirus and/or the COVID-19 pandemic", and a value of 0 if it is another subject.

Output your results in JSON format with keys "article_id" and "about_covid", where the article ID and your answer are the values. 
Follow the example output format provided. Do not include any additional information or explanation. \n

Example Output (JSON format):
{{
    "article_id": "2351150",
    "about_covid": "1"
}}
"""

In [ ]:
one_shot_example =[
("""
Read the following article with the ID 2327099: 'Sinds gisteren zijn 155 nieuwe mensen positief getest op het coronavirus. Dat brengt het totaal op 959 patiënten, meldt het RIVM.\n Er zijn twee nieuwe mensen aan de gevolgen van het virus overleden. Het gaat om ouderen met onderliggend lijden. In totaal staat het dodental in Nederland nu op twaalf.\n Gisteren werden 190 nieuwe patiënten gemeld. Volgens het RIVM is de daling van vandaag het gevolg van het veranderde testbeleid.\n "Sinds 12 maart worden mensen met milde klachten niet meer getest omdat er een landelijke maatregel is om thuis te blijven bij de eerste klachten. Daarnaast wordt er meer getest onder risicogroepen", zegt de instantie.\n Op 12 maart riep de overheid Nederlanders op om sociale contacten te vermijden. Het is volgens het RIVM nog te vroeg om het effect daarvan te zien.\n In het hele land was het vandaag op allerlei plekken veel rustiger dan anders op zaterdag:\n De cijfers zeggen niet alles over het totale aantal besmettingen in Nederland. Alleen ernstig zieke mensen worden nog getest, zodat er voldoende testmateriaal voor riskante gevallen beschikbaar blijft. Daarnaast zijn er volgens de GGD waarschijnlijk veel mensen met milde klachten die thuis uitzieken en zich niet melden.' \n
This article falls under the categories: 'Binnenland' and contains the keywords: 'virus, besmettingen, RIVM, corona'.

Take a moment to understand the article. 
Remember, for a topic to be a main topic of the news article, it should be discussed in the majority of the article. 

Based on the information provided, determine if the main topic of this news article is the "Coronavirus and/or the COVID-19 pandemic" or another subject. 
Assign a value of 1 if the main topic is the "Coronavirus and/or the COVID-19 pandemic", and a value of 0 if it is another subject.

Output your results in JSON format with keys "article_id" and "about_covid", where the article ID and your answer are the values. 
Follow the example output format provided. Do not include any additional information or explanation. \n

Example Output (JSON format):
{{
    "article_id": "2351150",
    "about_covid": "1"
}}
""", 
"""
{{
    "article_id": "2327099",
    "about_covid": "1"
}}
""")]

In [ ]:
few_shot_examples = [
("""
Read the following article with the ID 2327099: 'Sinds gisteren zijn 155 nieuwe mensen positief getest op het coronavirus. Dat brengt het totaal op 959 patiënten, meldt het RIVM.\n Er zijn twee nieuwe mensen aan de gevolgen van het virus overleden. Het gaat om ouderen met onderliggend lijden. In totaal staat het dodental in Nederland nu op twaalf.\n Gisteren werden 190 nieuwe patiënten gemeld. Volgens het RIVM is de daling van vandaag het gevolg van het veranderde testbeleid.\n "Sinds 12 maart worden mensen met milde klachten niet meer getest omdat er een landelijke maatregel is om thuis te blijven bij de eerste klachten. Daarnaast wordt er meer getest onder risicogroepen", zegt de instantie.\n Op 12 maart riep de overheid Nederlanders op om sociale contacten te vermijden. Het is volgens het RIVM nog te vroeg om het effect daarvan te zien.\n In het hele land was het vandaag op allerlei plekken veel rustiger dan anders op zaterdag:\n De cijfers zeggen niet alles over het totale aantal besmettingen in Nederland. Alleen ernstig zieke mensen worden nog getest, zodat er voldoende testmateriaal voor riskante gevallen beschikbaar blijft. Daarnaast zijn er volgens de GGD waarschijnlijk veel mensen met milde klachten die thuis uitzieken en zich niet melden.' \n
This article falls under the categories: 'Binnenland' and contains the keywords: 'virus, besmettingen, RIVM, corona'.

Take a moment to understand the article. 
Remember, for a topic to be a main topic of the news article, it should be discussed in the majority of the article. 

Based on the information provided, determine if the main topic of this news article is the "Coronavirus and/or the COVID-19 pandemic" or another subject. 
Assign a value of 1 if the main topic is the "Coronavirus and/or the COVID-19 pandemic", and a value of 0 if it is another subject.

Output your results in JSON format with keys "article_id" and "about_covid", where the article ID and your answer are the values. 
Follow the example output format provided. Do not include any additional information or explanation. \n

Example Output (JSON format):
{{
    "article_id": "2351150",
    "about_covid": "1"
}}
""", 
"""
{{
    "article_id": "2327099",
    "about_covid": "1"
}}
"""),
("""
Read the article with ID 2446472: 'De wachtrijen in de langdurige zorg zijn in juli fors gestegen. De toename was afgelopen jaar niet eerder zo groot, meldt de Nederlandse Zorgautoriteit (NZa) in haar maandelijkse overzicht.\n Het aantal mensen dat moet wachten op zorg vanuit de Wet langdurige zorg (Wlz) steeg van 21.653 op 1 juli naar 23.497 in augustus. De toename deed zich voor in alle drie de sectoren: verpleeg- en verzorgingstehuizen, gehandicaptenzorg en langdurige geestelijke gezondheidszorg. De NZa weet geen oorzaak voor de plotselinge stijging.\n De afgelopen tijd is het ook veel gegaan over het ziekteverzuim in de zorg. De NZa ziet dat die nog steeds in alle sectoren hoog is, maar er is nu wel een lichte daling zichtbaar in het kortdurend ziekteverzuim.\n Het langdurige ziekteverzuim is gelijk gebleven. Het ziekteverzuim in combinatie met personeelstekorten baart de zorgtoezichthouder zorgen, met name in de langdurige zorg. Er zijn vaak genoeg bedden beschikbaar, maar er is niet altijd genoeg personeel om de langdurige zorg te leveren.' \n
This article falls under the categories: 'Binnenland' and contains the keywords: 'verpleeg- en verzorgingshuizen, Wet Langdurige Zorg, langdurige zorg, ziekenhuiszorg'.

Take a moment to understand the article. 
Remember, for a topic to be a main topic of the news article, it should be discussed in the majority of the article. 

Based on the information provided, determine if the main topic of this news article is the "Coronavirus and/or the COVID-19 pandemic" or another subject. 
Assign a value of 1 if the main topic is the "Coronavirus and/or the COVID-19 pandemic", and a value of 0 if it is another subject.

Output your results in JSON format with keys "article_id" and "about_covid", where the article ID and your answer are the values. 
Follow the example output format provided. Do not include any additional information or explanation. \n

Example Output (JSON format):
{{
    "article_id": "2351150",
    "about_covid": "1"
}}
""",
"""
{{
    "article_id": "2446472",
    "about_covid": "0"
}}
""")
]

In [ ]:
not_covid[not_covid['article_id'] == 2399596].Text.values

In [ ]:
not_covid.head()

# About Covid

## Zero-Shot Classifier About_Covid

In [ ]:
zero_shot_prompt = zero_shot_prompt_messages(system_prompt, input_prompt, main_prompt)
print(zero_shot_prompt)

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["article_id", "text", "category", "keywords"],
    template=zero_shot_prompt
)

In [ ]:
chain_one = LLMChain(llm = llm, prompt = prompt_template, output_key="article_id, about_covid")
chain_one

In [ ]:
generated_text_zeroshot = []

In [ ]:
%%time
torch.manual_seed(0)

for index, row in df.iterrows():  
    article_id = row['article_id']
    text = row['Text']
    category = row['Category']
    keywords = row['Keywords']


    input_variables = {
            "article_id": article_id,
            "text": text,
            "category": category,
            "keywords": keywords
        }
    # Generate text using the chain
    generated_text = chain_one.run(input_variables)
    print(generated_text)
    generated_text_zeroshot.append(generated_text)    

In [ ]:
generated_text_zeroshot[0:5]

In [ ]:
json_list = regex_extract_to_dataframe(generated_text_zeroshot)

print(len(json_list))
df_zeroshot = pd.DataFrame(json_list)
print(len(df_zeroshot))
df_zeroshot.head()

In [ ]:
df_zeroshot['about_covid'].value_counts()

In [ ]:
# rename about_covid to about_covid_pred
df_zeroshot = df_zeroshot.rename(columns={'about_covid': 'about_covid_pred'})
df_zeroshot['article_id'] = df_zeroshot['article_id'].astype(int)
df_zeroshot['about_covid_pred'] = df_zeroshot['about_covid_pred'].astype(int)

df_zeroshot.head()

In [ ]:
# get real article_id and about_covid
df_coded = df[['article_id', 'about_covid']]
df_coded.head()

In [ ]:
# merge
df_zeroshot_merged = pd.merge(df_coded, df_zeroshot, how='left', on="article_id")
print(len(df_zeroshot_merged))
df_zeroshot_merged.head()

# make pred int
# drop nan
df_zeroshot_merged = df_zeroshot_merged.dropna()
df_zeroshot_merged['about_covid_pred'] = df_zeroshot_merged['about_covid_pred'].astype(int)

In [ ]:
pd.crosstab(df_zeroshot_merged['about_covid'], df_zeroshot_merged['about_covid_pred'])

In [ ]:
print(classification_report(df_zeroshot_merged['about_covid'], df_zeroshot_merged['about_covid_pred']))
print("Cohen's Kappa:", cohen_kappa_score(df_zeroshot_merged['about_covid'], df_zeroshot_merged['about_covid_pred']))

In [ ]:
# write df to results 
df_zeroshot_merged.to_csv('data/nos_analysis/results/about_covid_starling_zeroshot_v4.csv', sep = ';', encoding = 'utf-8', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
# change df shape wher
humancoded = df_zeroshot_merged[['article_id', 'about_covid']]
humancoded['coder']='human'
humancoded
machinecoded = df_zeroshot_merged[['article_id', 'about_covid_pred']]
machinecoded['coder']='machine'
machinecoded.rename(columns={'about_covid_pred': 'about_covid'}, inplace=True)

df_zeroshot_merged_krip = pd.concat([humancoded, machinecoded])

In [ ]:
zeroshot_krip = simpledorff.calculate_krippendorffs_alpha_for_df(df_zeroshot_merged_krip,experiment_col='article_id',
                                                 annotator_col='coder',
                                                 class_col='about_covid')

print("Krippendorff's Alpha:", zeroshot_krip)

## One-Shot Classifier About_Covid

In [ ]:
one_shot_prompt = few_shot_prompt_messages(system_prompt, input_prompt, main_prompt, one_shot_example)
print(one_shot_prompt)

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["article_id", "text", "category", "keywords"],
    template=one_shot_prompt
)

In [ ]:
chain_2 = LLMChain(llm = llm, prompt = prompt_template, output_key="article_id, about_covid")
chain_2         

In [ ]:
generated_text_oneshot = []

In [ ]:
%%time
torch.manual_seed(0)
for index, row in df.iterrows():  
    article_id = row['article_id']
    text = row['Text']
    category = row['Category']
    keywords = row['Keywords']


    input_variables = {
            "article_id": article_id,
            "text": text,
            "category": category,
            "keywords": keywords
        }
    # Generate text using the chain
    generated_text = chain_2.run(input_variables)
    generated_text_oneshot.append(generated_text)    


In [ ]:
generated_text_oneshot[0:5]

In [ ]:
json_list = regex_extract_to_dataframe(generated_text_oneshot)

print(len(json_list))
df_oneshot = pd.DataFrame(json_list)
print(len(df_oneshot))
df_oneshot.head()

In [ ]:
df_oneshot['about_covid'].value_counts()

In [ ]:
# rename about_covid to about_covid_pred
df_oneshot = df_oneshot.rename(columns={'about_covid': 'about_covid_pred'})

df_oneshot.head()

In [ ]:
#change article_id to int
df_oneshot['article_id'] = df_oneshot['article_id'].astype(int)
df_coded['article_id'] = df_coded['article_id'].astype(int)

df_oneshot['about_covid_pred'] = df_oneshot['about_covid_pred'].astype(int)
df_coded['about_covid'] = df_coded['about_covid'].astype(int)


In [ ]:
# get real article_id and about_covid
df_coded = df[['article_id', 'about_covid']]
df_coded.head()

# merge df_zeroshot_v2 with df on index
df_oneshot_merged = pd.merge(df_coded, df_oneshot, how='left', on="article_id")
print(len(df_oneshot_merged))
df_oneshot_merged.head()

In [ ]:
pd.crosstab(df_oneshot_merged['about_covid'], df_oneshot_merged['about_covid_pred'])

In [ ]:
print(classification_report(df_oneshot_merged['about_covid'], df_oneshot_merged['about_covid_pred']))
print("Cohen's Kappa:", cohen_kappa_score(df_oneshot_merged['about_covid'], df_oneshot_merged['about_covid_pred']))

In [ ]:
# change df shape wher
humancoded = df_oneshot_merged[['article_id', 'about_covid']]
humancoded['coder']='human'
humancoded
machinecoded = df_oneshot_merged[['article_id', 'about_covid_pred']]
machinecoded['coder']='machine'
machinecoded.rename(columns={'about_covid_pred': 'about_covid'}, inplace=True)

df_oneshot_merged_krip = pd.concat([humancoded, machinecoded])

In [ ]:
oneshot_krip = simpledorff.calculate_krippendorffs_alpha_for_df(df_oneshot_merged_krip,experiment_col='article_id',
                                                 annotator_col='coder',
                                                 class_col='about_covid')

print("Krippendorff's Alpha:", oneshot_krip)

In [ ]:
# write df to results 
df_oneshot_merged.to_csv('data/nos_analysis/results/about_covid_starling_oneshot_v4.csv', sep = ';', encoding = 'utf-8', quoting=csv.QUOTE_NONNUMERIC)

## Few-Shot Classifier About_Covid

In [ ]:
few_shot_prompt = few_shot_prompt_messages(system_prompt, input_prompt, main_prompt, few_shot_examples)
print(few_shot_prompt)

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["article_id", "text", "category", "keywords"],
    template=few_shot_prompt
)

In [ ]:
chain_3 = LLMChain(llm = llm, prompt = prompt_template, output_key="article_id, about_covid")
chain_3

In [ ]:
generated_text_fewshot = []

In [ ]:
%%time
torch.manual_seed(0)
for index, row in df.iterrows():  
    article_id = row['article_id']
    text = row['Text']
    category = row['Category']
    keywords = row['Keywords']


    input_variables = {
            "article_id": article_id,
            "text": text,
            "category": category,
            "keywords": keywords
        }
    # Generate text using the chain
    generated_text = chain_3.run(input_variables)
    generated_text_fewshot.append(generated_text)    


In [ ]:
generated_text_fewshot[0:5]

In [ ]:
json_list = regex_extract_to_dataframe(generated_text_fewshot)

print(len(json_list))
df_fewshot = pd.DataFrame(json_list)
print(len(df_fewshot))
df_fewshot.head()

In [ ]:
df_fewshot['about_covid'].value_counts()

In [ ]:
# rename about_covid to about_covid_pred
df_fewshot = df_fewshot.rename(columns={'about_covid': 'about_covid_pred'})

df_fewshot.head()

In [ ]:
# get real article_id and about_covid
df_coded = df[['article_id', 'about_covid']]
df_coded.head()

# merge df_zeroshot_v2 with df on index
df_fewshot_merged = pd.merge(df_coded, df_fewshot, how='left', on="article_id")
print(len(df_fewshot_merged))
df_fewshot_merged.head()

In [ ]:
# is there nan
df_fewshot_merged.isnull().sum()

# drop if about_covid_pred is null
df_fewshot_merged=df_fewshot_merged[df_fewshot_merged['about_covid_pred'].notnull()]
df_fewshot_merged['about_covid_pred'] = df_fewshot_merged['about_covid_pred'].astype(int)

In [ ]:
pd.crosstab(df_fewshot_merged['about_covid'], df_fewshot_merged['about_covid_pred'])

In [ ]:
print(classification_report(df_fewshot_merged['about_covid'], df_fewshot_merged['about_covid_pred']))
print("Cohen's Kappa:", cohen_kappa_score(df_fewshot_merged['about_covid'], df_fewshot_merged['about_covid_pred']))

In [ ]:
# change df shape wher
humancoded = df_fewshot_merged[['article_id', 'about_covid']]
humancoded['coder']='human'
humancoded
machinecoded = df_fewshot_merged[['article_id', 'about_covid_pred']]
machinecoded['coder']='machine'
machinecoded.rename(columns={'about_covid_pred': 'about_covid'}, inplace=True)

df_fewshot_merged_krip = pd.concat([humancoded, machinecoded])

In [ ]:
fewshot_krip = simpledorff.calculate_krippendorffs_alpha_for_df(df_fewshot_merged_krip,experiment_col='article_id',
                                                 annotator_col='coder',
                                                 class_col='about_covid')

print("Krippendorff's Alpha:", fewshot_krip)

In [ ]:
# write df to results 
df_fewshot_merged.to_csv('data/nos_analysis/results/about_covid_starling_fewshot_v4.csv', sep = ';', encoding = 'utf-8', quoting=csv.QUOTE_NONNUMERIC)